In [1]:
from requests import get
from pandas import DataFrame, concat, to_datetime, to_timedelta, json_normalize, notna, isna
from time import time
import numpy as np
from dotenv import load_dotenv
import json, os
from datetime import datetime

In [2]:
fetchSaleData = True
fetchReturnData = True

lastUpdatedInDays = 1

In [3]:
# current time
currentTime = time()
timeToCheckAgainst = datetime.fromtimestamp(currentTime - lastUpdatedInDays*24*60*60)

if not os.path.exists('../data files'):
    os.makedirs('../data files')

print('Checking for sale data...')
if os.path.exists("../data files/rawSaleData.json"):
    saleDataTime = datetime.fromtimestamp(os.path.getmtime('../data files/rawSaleData.json'))
    print('Found sale data from:', saleDataTime.strftime('%c'))
    
    # if older than 'lastUpdatedInDays' days, fetch again
    if saleDataTime <= timeToCheckAgainst:
        print('Locally saved data is older than {} hours!'.format(lastUpdatedInDays*24))
        print('Have to fetch again...')
        fetchSaleData = True
    else:
        print('Will be using this locally saved data...')
        fetchSaleData = False
else:
    print('No local data found!')
    print('Have to fetch again...')
    fetchSaleData = True

print('Checking for return data...')
if os.path.exists("../data files/rawReturnData.json"):
    returnDataTime = datetime.fromtimestamp(os.path.getmtime('../data files/rawReturnData.json'))
    print('Found return data from:', returnDataTime.strftime('%c'))
    
    # if older than 'lastUpdatedInDays' days, fetch again
    if returnDataTime <= timeToCheckAgainst:
        print('Locally saved data is older than {} hours!'.format(lastUpdatedInDays*24))
        print('Have to fetch again...')
        fetchReturnData = True
    else:
        print('Will be using this locally saved data...')
        fetchReturnData = False
else:
    print('No local data found!')
    print('Have to fetch again...')
    fetchReturnData = True

Checking for sale data...
Found sale data from: Wed Jul 28 18:04:58 2021
Locally saved data is older than 24 hours!
Have to fetch again...
Checking for return data...
Found return data from: Wed Jul 28 18:13:07 2021
Locally saved data is older than 24 hours!
Have to fetch again...


In [4]:
load_dotenv()

domain = os.getenv("URL")

if fetchSaleData:
    print('Getting Sale data... Please wait!')
    startTime = time()

    # get sale data
    response = get('https://' + domain + 'sale')
    saleData = response.json()

    print('Took', round(time() - startTime, 2), 'seconds to get Sale data.')
    
    # save locally for later reuse
    with open('../data files/rawSaleData.json', 'w') as fp:
        json.dump(saleData, fp, indent=4)
else:
    # use locally saved data
    with open('../data files/rawSaleData.json', 'r') as fp:
        saleData = json.load(fp)

Getting Sale data... Please wait!
Took 22.92 seconds to get Sale data.


In [5]:
if fetchReturnData:
    print('Getting Return data... Please wait!')
    startTime = time()

    # get return data
    response = get('https://' + domain + 'return')
    returnData = response.json()

    print('Took', round(time() - startTime, 2), 'seconds to get Return data.')
    
    # save locally for later reuse
    with open('../data files/rawReturnData.json', 'w') as fp:
        json.dump(returnData, fp, indent=4)
else:
    # use locally saved data
    with open('../data files/rawReturnData.json', 'r') as fp:
        returnData = json.load(fp)

Getting Return data... Please wait!
Took 14.53 seconds to get Return data.


## Taking Sale Data

In [6]:
# build dataframe
saleDF = DataFrame.from_dict(saleData, orient='columns')

# drop columns
saleDF = saleDF.drop(['scanned_at', 'scanned_at_melbourne_date_time', 'return_id'], axis=1)

# rename
saleDF = saleDF.rename(columns={"scanned_at_melbourne_date": "sale_date", "scanned_at_melbourne_time": "sale_time", "id": "sale_id"})

# merging date and time, converting new column to datetime64 datatype
saleDF['date_time_concat'] = saleDF['sale_date'] + ' ' + saleDF['sale_time']
saleDF['date_time'] = to_datetime(saleDF['date_time_concat'], dayfirst=True)
saleDF = saleDF.drop(['sale_date', 'sale_time', 'date_time_concat'], axis=1)

# filter to get records post 22nd Jan EDIT post Feb 3rd 2021
saleDF = saleDF[saleDF['date_time'] >= '2021/02/03 00:00:00']
saleDF = saleDF.sort_values(by='date_time', ascending = True).reset_index(drop=True)

saleDF.head()

,sale_id,cup_id,cafe_id,date_time
0,6997,3517464954,101,2021-04-27 14:35:35
1,6998,1918660986,101,2021-04-27 14:55:50
2,6999,6762937484,101,2021-04-29 07:29:58
3,7000,1918660986,101,2021-04-29 07:30:07
4,7001,4405493099,101,2021-04-29 07:30:20


In [7]:
saleDF['week'] =  saleDF.date_time.dt.isocalendar().week
saleDF.head()

,sale_id,cup_id,cafe_id,date_time,week
0,6997,3517464954,101,2021-04-27 14:35:35,17
1,6998,1918660986,101,2021-04-27 14:55:50,17
2,6999,6762937484,101,2021-04-29 07:29:58,17
3,7000,1918660986,101,2021-04-29 07:30:07,17
4,7001,4405493099,101,2021-04-29 07:30:20,17


In [8]:
groupDF = saleDF.groupby(by=['cup_id', 'week']).size().reset_index(name='occurrence_per_week')
#groupDF.loc[groupDF['occurrence_per_week'] > 1, :]
groupDF.head()

,cup_id,week,occurrence_per_week
0,1017492318,21,1
1,1034777146,19,1
2,1045514637,21,1
3,1052456801,18,2
4,1133606645,17,1


In [9]:
#groupDF.groupby(by='cup_id')['occurrence_per_week'].apply(list).reset_index(name='new')
df = groupDF.pivot(index='cup_id', columns='week', values='occurrence_per_week').reset_index()
df.columns.name = None
df.head()

,cup_id,17,18,19,20,21,27,31
0,1017492318,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,1034777146,NaN,NaN,1.0,NaN,NaN,NaN,NaN
2,1045514637,NaN,NaN,NaN,NaN,1.0,NaN,NaN
3,1052456801,NaN,2.0,NaN,NaN,NaN,NaN,NaN
4,1133606645,1.0,NaN,NaN,2.0,NaN,NaN,NaN


In [10]:
jsonObject = {}
nodesArray = []
linksArray = []

def updateNode(flag, weekStart, weekEnd=0):
            
    # update node for a cup's single use in all weeks
    if flag == 0:
        # find the existing node
        for index in range(len(nodesArray)):
            node = nodesArray[index]
            if node['week'] == weekStart:
                nodesArray[index]['single'] += 1
                break

    # update node for a cup's use in the same week (i.e used 2, 3 or more times in that week)
    elif flag == 1:
        # find the existing node
        for index in range(len(nodesArray)):
            node = nodesArray[index]
            if node['week'] == weekStart:
                # if not the first time, add 1
                if node['self'] != 0:
                    nodesArray[index]['self'] += 1
                    
                # if first time, add 2
                else:
                    nodesArray[index]['self'] += 2
                break

    # update nodes for a cup's use in 'start' and 'end' week
    else:
        # find the existing nodes
        for index in range(len(nodesArray)):
            node = nodesArray[index]
            if node['week'] == weekStart or node['week'] == weekEnd:
                if 'other' in node:
                    nodesArray[index]['other'] += 1
                else:
                    nodesArray[index]['other'] = 1

In [11]:
def populateNodes(df):
    for node in list(df.columns[1:]):
        nodesArray.append({
            'week': node,
            'single': 0,
            'self': 0,
            'other': 0
        })
        
def populateLinks(df):
    #count = 0
    for index, row in df.iterrows():
        #print(row)
        start = None
        end = None
        for week in list(df.columns[1:]):
            if notna(row[week]):
                for uses in range(0, int(row[week])):
                    if start == None:
                        start = week
                    else:
                        end = week

                        # flag
                        found = False

                        # search for an existing link between start and end
                        for index in range(len(linksArray)):
                            link = linksArray[index]

                            # if link found, update count
                            if link['source'] == start and link['target'] == end:
                                found = True
                                linksArray[index]['count'] += 1
                                break

                        # if link is not found, create new link
                        if found == False:
                            if start != end:
                                linksArray.append({
                                    'source': start,
                                    'target': end,
                                    'count' : 1
                                })
                            else:
                                linksArray.append({
                                    'source': start,
                                    'target': end,
                                    'count' : 2
                                })

                        if start == end:
                            # update node with week = start for self use denoted by flag = 1
                            updateNode(1, start)
                        else:
                            # update both nodes with week = start and week = end for 'start' (in past) week use 
                            # denoted by flag = 2
                            updateNode(2, start, end)

                        start = week
        if end == None:
            # update node with week = start for single use denoted by flag = 0
            updateNode(0, start)

        #count += 1
        #if count == 10:
        #  break
#print('nodes', nodesArray)
#print('links', linksArray)
#print('singleuse', singleUse)

In [12]:
populateNodes(df)

populateLinks(df)

In [13]:
# sorting in descending order so that bigger arc widths dont obscure smaller arc widths
linksArray = sorted(linksArray, key=lambda d: (-d['count']))

In [14]:
jsonObject['nodes'] = nodesArray
jsonObject['links'] = linksArray

with open('../data files/arcSaleData.json', 'w') as fp:
    json.dump(jsonObject, fp, indent=4)

## Taking Return Data

In [15]:
# build dataframe
returnDF = DataFrame.from_dict(json_normalize(returnData), orient='columns')

# dropping columns 
returnDF = returnDF.drop(['scanned_at', 'bin_id', 'cafe_id', 'dishwasher_id', 'id'], axis=1)

# renaming columns
returnDF = returnDF.rename(columns={"scanned_at_melbourne_date_time.date": "return_date", "scanned_at_melbourne_date_time.time": "return_time"})

# merging date and time, converting new column to datetime64 datatype
returnDF['date_time_concat'] = returnDF['return_date'] + ' ' + returnDF['return_time']
returnDF['date_time'] = to_datetime(returnDF['date_time_concat'], dayfirst=True)
returnDF = returnDF.drop(['return_date', 'return_time', 'date_time_concat'], axis=1)

# filter to get records post 22nd Jan
returnDF = returnDF[returnDF['date_time'] >= '2020/01/22 00:00:00']
returnDF = returnDF.sort_values(by='date_time', ascending = True)

returnDF.head()

,cup_id,date_time
6243,3815501729,2020-01-22 14:55:40
6244,2397660595,2020-01-22 14:55:42
6245,4897580664,2020-01-22 14:55:43
6246,2486468654,2020-01-22 14:55:44
6247,4498550854,2020-01-22 14:55:45


In [16]:
returnDF['week'] =  returnDF.date_time.dt.isocalendar().week
returnDF.head()

,cup_id,date_time,week
6243,3815501729,2020-01-22 14:55:40,4
6244,2397660595,2020-01-22 14:55:42,4
6245,4897580664,2020-01-22 14:55:43,4
6246,2486468654,2020-01-22 14:55:44,4
6247,4498550854,2020-01-22 14:55:45,4


In [17]:
groupDF = returnDF.groupby(by=['cup_id', 'week']).size().reset_index(name='occurrence_per_week')
#groupDF.loc[groupDF['occurrence_per_week'] > 1, :]
groupDF.head()

,cup_id,week,occurrence_per_week
0,1008415089,4,1
1,1008415089,5,1
2,1008415089,6,1
3,1008415089,7,1
4,1008415089,9,2


In [18]:
#groupDF.groupby(by='cup_id')['occurrence_per_week'].apply(list).reset_index(name='new')
df = groupDF.pivot(index='cup_id', columns='week', values='occurrence_per_week').reset_index()
df.columns.name = None
df.head()

,cup_id,4,5,6,7,8,9,10,11,17,18,19,20,21,28,31
0,1008415089,1.0,1.0,1.0,1.0,NaN,2.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1012822375,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1017030777,NaN,NaN,NaN,2.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1017492318,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,1019325282,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
jsonObject = {}
nodesArray = []
linksArray = []

populateNodes(df)

populateLinks(df)

In [20]:
# sorting in descending order so that bigger arc widths dont obscure smaller arc widths
linksArray = sorted(linksArray, key=lambda d: (-d['count']))

In [21]:
jsonObject['nodes'] = nodesArray
jsonObject['links'] = linksArray

with open('../data files/arcReturnData.json', 'w') as fp:
    json.dump(jsonObject, fp, indent=4)